# Applied Data Science Capstone
___
## Week 3

#### This is part of the course "Applied Data Science Capstone" from "IBM Data Science Professional Certificate"

In [1]:
import pandas as pd
import numpy as np
import requests

print('Ready to start')

Ready to start


# Question 1: Extract the neighbourhood
---
We will scrape the list of postal codes of Canada, along with boroughs and neighbourhoods  
We will create a data frame, discarding the 'Not assigned' records

In [2]:
link0 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(link0)
df = df[0]

In [3]:
print('Shape:', df.shape[0],'rows and', df.shape[1], 'columns')

Shape: 180 rows and 3 columns


In [4]:
for x in df.columns.tolist(): print(x+':', df[x].nunique())

Postal Code: 180
Borough: 11
Neighbourhood: 100


In [5]:
df = df[df['Borough'] != 'Not assigned']
print(df.shape, '\n')
print('The shape after cleaning the non assigned records:\n*', df.shape[0],'rows\n*', df.shape[1], 'columns')

(103, 3) 

The shape after cleaning the non assigned records:
* 103 rows
* 3 columns


# Question 2: Extracting the coordinates of each neighbourhood
---
* We will get the coordinates for each postal code from the link provided
* Unfortunately, the geo-coder is not installed in the Watson Studio and  
its installation is time consuming.  
* We will create a data frame, with postal codes along with their longitude and latitude  
* We will the long-lat data to our original data frame with the neighbourhoods

### Step 1: Read the data from the link and place it in a data frame

In [7]:
link1 = 'http://cocl.us/Geospatial_data'

df0 = pd.read_csv(link1)
df0.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Step 2: Join the two dataframes based on Postac Code

In [8]:
df1 = df.set_index('Postal Code').join(df0.set_index('Postal Code'), how = 'left')
df1.reset_index(inplace = True)
print('columns:', df1.columns.tolist())
print('shape:', df1.shape)

df1.head(6)

columns: ['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']
shape: (103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242


# Question 3: Exploring and clustering the neighbourhoods
---
* We will perform the analysis in Neighbourhoods in Boroughs that contain 'Toronto' in their name
* We will cluster them

### Step 1: Create a folium map for the neighbourhoods

In [35]:
tor_coords = [43.651070, -79.347015]
print(tor_coords)

[43.65107, -79.347015]


#### The following cell contains code to install folium 0.5.0.  
It is commented in order to be skipped

In [44]:
# !conda install -c conda-forge folium=0.5.0 --yes

In [134]:
import folium

tor_map = folium.Map(location = tor_coords, zoom_start = 10,width = 1080, height = 585)

for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(tor_map)  
    
tor_map

### Step 2: Identify Boroughs with 'Toronto' in their name
* we create a list which states whether a postal code has a borough containint toronto (True or False)
* we add this list to the data frame
* we extract a new data frame containing only records with 'Toronto' in Borough name

In [55]:
TorontoBoroughs = []

for i in range(df1.shape[0]): TorontoBoroughs.append('Toronto' in df1.loc[i, 'Borough'])
    
df1['Toronto_Boroughs'] = TorontoBoroughs

df_toronto = df1[df1['Toronto_Boroughs'] == True]
df_toronto = df_toronto.iloc[:, 0:5]

print(df_toronto.shape)
df_toronto.head()

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Step 3: Create a folium map for these neighbourhoods

In [135]:
tor_map_selected = folium.Map(location = tor_coords, zoom_start = 11,width = 1080, height = 585)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.6,
        parse_html = False).add_to(tor_map_selected)  
    
tor_map_selected

### Step 4: Extract venues for each neighbourhood from Foursquare

#### Step 4.1: Our credentials (not shown), along with limit and radius options for fetching data from 

In [2]:
# The code was removed by Watson Studio for sharing.

In [60]:
LIMIT = 100
radius = 500

#### Step 4.2: Define a function that will retrieve the data
* it will create a url based on the neighbourhood name, lat and long, along with credentials
* it will fetch the data in json format
* it will create a data frame with the records

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Step 4.3: Define a function that will retrieve the venue categories
* the category list might have multiple categories, we will fetch the first
* it might be possible for a venue to have no category, so the funtion will return None

In [62]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [63]:
df_toronto.columns.tolist()

['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

#### Step 4.4: Extract the venues with the aformentioned functions

In [64]:
df_venues = getNearbyVenues(names = df_toronto['Neighbourhood'],
                                   latitudes = df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude']
                                  )

df_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [65]:
df_venues.shape

(1647, 7)

#### Venues per Neighborhood

In [114]:
df_venues.groupby('Neighborhood').count().head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [68]:
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

There are 234 uniques categories.


### Step 5: Analyse each neighbourhood 

#### Step 5.1: One hot the venue categories, we will study category frequency per neighbourhood

In [103]:
df_venues_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

df_venues_onehot['Neighborhood'] = df_venues['Neighborhood'] 

index1 = df_venues_onehot.columns.tolist().index('Neighborhood')
temp_list = df_venues_onehot.columns.tolist()[:]
print(len(temp_list))
temp_list.remove('Neighborhood')

arrange_columns = [df_venues_onehot.columns[index1]] + temp_list
df_venues_onehot = df_venues_onehot[arrange_columns]

df_venues_onehot.head(10)

234


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
df_venues_onehot.shape

(1647, 234)

#### Step 5.2: Group per neighbourhood

In [126]:
df_venues_grouped = df_venues_onehot.groupby('Neighborhood').mean().reset_index()
df_venues_grouped.head(10)

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.017544,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.080000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.052632
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.014925,0.0,0.0,0.014925,0.0,0.0,0.014925
5,Christie,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
6,Church and Wellesley,0.012987,0.0000,0.0000,0.000,0.000,0.000,0.012987,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.0,0.0,...,0.000000,0.0,0.0,0.020000,0.0,0.0,0.010000,0.0,0.0,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.055556,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [109]:
df_venues_grouped.shape

(39, 234)

#### Step 5.3: Let's print each neighborhood along with the top 5 most common venues

In [111]:
num_top_venues = 5

for hood in df_venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_venues_grouped[df_venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.11
1    Cocktail Bar  0.04
2     Cheese Shop  0.04
3  Farmers Market  0.04
4          Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Yoga Studio  0.08
2          Bakery  0.08
3     Coffee Shop  0.08
4  Breakfast Spot  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1  Gym / Fitness Center  0.05
2               Brewery  0.05
3                   Spa  0.05
4            Smoke Shop  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3               Plane  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                 venue  freq
0          Coffee S

#### Step 5.4: Let's put this information into a data frame
* we will create a function to create the data frames
* we will use this function 

In [112]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = df_venues_grouped['Neighborhood']

for ind in np.arange(df_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Eastern European Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Miscellaneous Shop


### Step 6: Cluster analysis

#### Step 6.1: We will cluster the neighborhoods based on the venue category frequency

In [121]:
from sklearn.cluster import KMeans

kclusters = 4

df_venues_grouped_clustering = df_venues_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(df_venues_grouped_clustering)

kmeans.labels_[:] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Step 6.2: combine the original data with the cluster number

In [124]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_toronto_merged = df_toronto

df_toronto_merged = df_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

df_toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Park
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Bookstore,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Breakfast Spot,Park,Moroccan Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


#### Step 6.3: Create a map visualizing the clusters

In [128]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [136]:
map_clusters = folium.Map(location = tor_coords, zoom_start=11, width = 1080, height = 585)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto_merged['Latitude'], df_toronto_merged['Longitude'], df_toronto_merged['Neighbourhood'], df_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

#### Step 6.4: An example of the big cluster

In [132]:
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 0, df_toronto_merged.columns[[1] + list(range(5, df_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
4,Downtown Toronto,0,Coffee Shop,Diner,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Park
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Bookstore,Ramen Restaurant
15,Downtown Toronto,0,Café,Coffee Shop,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Breakfast Spot,Park,Moroccan Restaurant
19,East Toronto,0,Trail,Health Food Store,Pub,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
20,Downtown Toronto,0,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Eastern European Restaurant
24,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Miscellaneous Shop
25,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Nightclub,Italian Restaurant,Diner,Candy Store,Baby Store,Coffee Shop
30,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Bar,Clothing Store,Hotel,Gym,Thai Restaurant,Steakhouse,Cosmetics Shop
31,West Toronto,0,Bakery,Pharmacy,Brewery,Supermarket,Bank,Middle Eastern Restaurant,Café,Music Venue,Pizza Place,Park
